# ATRX Mutation Trans Effect on Transcriptomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "TP53"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
omics = "transcriptomics"
#omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Transcriptomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

# Show interacting protein list
print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
RPA1
AURKA
MDM2
SIRT1
CREBBP
EP300
CDK2
TP53
ATM
BCL2L1
TP53BP2
CCNG1
MDM4
PTEN
TP53BP1
CHEK2
FZR1
MAPK8
KDM1A
BCL6
CDKN1A
HIPK2
CHEK1
CDKN2A
BRCA1
AKT1
DVL2
BLM
ATR
TFAP2A
EPHA3


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, flatten=True)
protdf = protdf[protdf.Sample_Status == 'Tumor'] # drop Normal samples

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)
        

Doing t-test comparisons

                                  Comparison       P_Value
0  CDKN2A_transcriptomics_ENSG00000147889.15  1.127793e-07





# All Proteins: Transcriptomics 

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten=True) # flatten for specific column names
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create binary column 
    protdf['Label'] = np.where(
                protdf[gene+'_Mutation_Status'] == 'Wildtype_Tumor', 'Wildtype', 'Mutated')

    # Format the dataframe correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  TP53


Doing t-test comparisons

                                       Comparison       P_Value
0          AEN_transcriptomics_ENSG00000181026.14  8.562056e-10
1        EDA2R_transcriptomics_ENSG00000131080.13  8.800930e-10
2         DDB2_transcriptomics_ENSG00000134574.10  1.176977e-08
3        PHLDA3_transcriptomics_ENSG00000174307.6  3.052271e-08
4          DRG2_transcriptomics_ENSG00000108591.8  8.233337e-08
5   RP11-37C7.2_transcriptomics_ENSG00000259432.1  1.002485e-07
6       CDKN2A_transcriptomics_ENSG00000147889.15  1.127793e-07
7       DCAF12_transcriptomics_ENSG00000198876.11  1.445258e-07
8         ZFAT_transcriptomics_ENSG00000066827.14  1.489154e-07
9        GPR176_transcriptomics_ENSG00000166073.7  1.589371e-07
10     PIP5KL1_transcriptomics_ENSG00000167103.10  7.240112e-07





# Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,TP53,CDKN2A_transcriptomics_ENSG00000147889.15,True,1.127793e-07
0,Gbm,TP53,AEN_transcriptomics_ENSG00000181026.14,False,8.562056e-10
1,Gbm,TP53,EDA2R_transcriptomics_ENSG00000131080.13,False,8.800930e-10
2,Gbm,TP53,DDB2_transcriptomics_ENSG00000134574.10,False,1.176977e-08
3,Gbm,TP53,PHLDA3_transcriptomics_ENSG00000174307.6,False,3.052271e-08
4,Gbm,TP53,DRG2_transcriptomics_ENSG00000108591.8,False,8.233337e-08
5,Gbm,TP53,RP11-37C7.2_transcriptomics_ENSG00000259432.1,False,1.002485e-07
6,Gbm,TP53,CDKN2A_transcriptomics_ENSG00000147889.15,False,1.127793e-07
7,Gbm,TP53,DCAF12_transcriptomics_ENSG00000198876.11,False,1.445258e-07
8,Gbm,TP53,ZFAT_transcriptomics_ENSG00000066827.14,False,1.489154e-07


### Write Significant Comparisons (if any) to Shared CSV file

In [10]:
'''
existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"\nexisting_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"